In [1]:
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras_vggface.utils import preprocess_input
from keras.models import Model
import numpy as np
from keras import Sequential
from keras import Input
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,GlobalMaxPooling2D,MaxPooling2D
from keras import optimizers
import tensorflow as tf
from keras import backend as K

from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
def load_file(examples_list_file):
    lines = np.genfromtxt(examples_list_file, delimiter="", dtype=[('col1', 'S120'), ('col2', 'i8')])
    #f=open(train_file)
    #lines=f.readlines()
    examples = []
    labels = []
    for example, label in lines:
        examples.append(example)
        labels.append(label)
    return np.asarray(examples), np.asarray(labels)

In [3]:
conv_base = VGGFace(model = 'vgg16', weights = 'vggface',input_shape=None,include_top=False)
conv_base.trainable = False
#print(conv_base.summary())
x_input = Input(shape=(150,150,3))
x =  (conv_base)(x_input)
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation= 'relu')(x)
predictions = Dense(5,activation='softmax')(x)
#setup_to_finetune(model)
model = Model(inputs=x_input,outputs=predictions)
#model.summary()
opt = optimizers.Adam(lr=0.00001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [5]:
acclist=[]
for i in range(26):
    print("*******round%02d**********"%(i+1))
    base_dir = 'k-fold'
    train_files=base_dir+"/path_txt/train%02d.txt"%(i+1)
    valid_files=base_dir+"/path_txt/valid%02d.txt"%(i+1)
    #path_tfrecords_train="k-fold/tfrecords/train%02d.tfrecords"%(i+1)
    #path_tfrecords_valid="k-fold/tfrecords/test%02d.tfrecords"%(i+1)
    examples_train,labels_train=load_file(train_files)
    examples_valid,labels_valid=load_file(valid_files)
    img_train=np.empty((len(examples_train),150,150,3),dtype="float32")
    print (len(examples_train))
    for i in range(5):
        images = image.load_img(examples_train[i], target_size=(150, 150,3))  
        x = image.img_to_array(images)
        #x = np.expand_dims(x, axis=0) 
        img_train[i,:,:,:] = x
    
    img_valid=np.empty((len(examples_valid),150,150,3),dtype="float32")
    print (len(examples_valid))
    for i in range(5):
        images = image.load_img(examples_train[i], target_size=(150, 150,3))  
        x = image.img_to_array(images)
        #x = np.expand_dims(x, axis=0) 
        img_valid[i,:,:,:] = x
        
        
    #print(labels_train)
    categorical_labels = to_categorical(labels_train, num_classes=6)[:,1:6]
    valid_labels=to_categorical(labels_valid,num_classes=6)[:,1:6]
    
    history=model.fit(img_train,categorical_labels,batch_size=None,epochs=2,
          verbose=1, callbacks=None, validation_split=0.0,
          validation_data=(img_valid,valid_labels), shuffle=True, 
          class_weight=None, sample_weight=None, 
          initial_epoch=0, steps_per_epoch=None, 
          validation_steps=None)
    acclist.append(history.history['val_acc'])


*******round01**********
16226
497
Train on 16226 samples, validate on 497 samples
Epoch 1/2
16226/16226 [==============================] - 40s 2ms/step - loss: 1.6128 - acc: 0.3752 - val_loss: 1.5904 - val_acc: 0.5714
Epoch 2/2
16226/16226 [==============================] - 39s 2ms/step - loss: 1.6110 - acc: 0.3752 - val_loss: 1.5863 - val_acc: 0.5714
*******round02**********
15627
1096
Train on 15627 samples, validate on 1096 samples
Epoch 1/2
15627/15627 [==============================] - 40s 3ms/step - loss: 1.6012 - acc: 0.3857 - val_loss: 1.6637 - val_acc: 0.3139
Epoch 2/2
15627/15627 [==============================] - 39s 2ms/step - loss: 1.5924 - acc: 0.3857 - val_loss: 1.6715 - val_acc: 0.3139
*******round03**********
16251
472
Train on 16251 samples, validate on 472 samples
Epoch 1/2
16251/16251 [==============================] - 40s 2ms/step - loss: 1.5875 - acc: 0.3834 - val_loss: 1.7208 - val_acc: 0.3008
Epoch 2/2
16251/16251 [==============================] - 39s 2ms/step

16390
333
Train on 16390 samples, validate on 333 samples
Epoch 1/2
16390/16390 [==============================] - 41s 3ms/step - loss: 1.4857 - acc: 0.3789 - val_loss: 1.5014 - val_acc: 0.4865
Epoch 2/2
16390/16390 [==============================] - 40s 2ms/step - loss: 1.4857 - acc: 0.3789 - val_loss: 1.5026 - val_acc: 0.4865
*******round25**********
16271
452
Train on 16271 samples, validate on 452 samples
Epoch 1/2
16271/16271 [==============================] - 41s 3ms/step - loss: 1.4791 - acc: 0.3827 - val_loss: 1.7398 - val_acc: 0.3097
Epoch 2/2
16271/16271 [==============================] - 40s 2ms/step - loss: 1.4790 - acc: 0.3827 - val_loss: 1.7418 - val_acc: 0.3097
*******round26**********
15622
1101
Train on 15622 samples, validate on 1101 samples
Epoch 1/2
15622/15622 [==============================] - 41s 3ms/step - loss: 1.4910 - acc: 0.3869 - val_loss: 1.3415 - val_acc: 0.3025
Epoch 2/2
15622/15622 [==============================] - 40s 3ms/step - loss: 1.4907 - acc: 0.

In [6]:
acclist

[[0.5714285714285714, 0.5714285714285714],
 [0.31386861313868614, 0.31386861313868614],
 [0.30084745762711862, 0.30084745762711862],
 [0.32333333333333331, 0.32333333333333331],
 [0.65660685154975529, 0.65660685154975529],
 [0.19540229885057472, 0.19540229885057472],
 [0.43902439024390244, 0.43902439024390244],
 [0.69372693726937273, 0.69372693726937273],
 [0.24977127172918573, 0.24977127172918573],
 [0.99464668094218411, 0.99464668094218411],
 [0.58246073298429324, 0.58246073298429324],
 [0.06903765690376569, 0.06903765690376569],
 [0.64935064935064934, 0.64935064935064934],
 [0.25448028673835127, 0.25448028673835127],
 [0.023809523809523808, 0.023809523809523808],
 [0.36990595611285265, 0.36990595611285265],
 [0.060901906090190611, 0.060901906090190611],
 [0.96527777777777779, 0.96527777777777779],
 [0.24130663856691253, 0.24130663856691253],
 [0.8766917293233083, 0.8766917293233083],
 [0.20472440944881889, 0.20472440944881889],
 [0.0, 0.0],
 [0.29298642533936653, 0.29298642533936653

In [7]:
import numpy as np
np.mean(acclist)

0.40108705439983122